# Chains in LangChain

## Outline
* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [ ]:
import datetime

current_date = datetime.datetime.now().date()

target_date = datetime.date(2024, 6, 12)

if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [ ]:
# Import pandas
import pandas as pd

In [ ]:
# Load dataset
df = pd.read_csv('l3_data.csv')
df.head()

## LLMChain

In [ ]:
from langchain_openai import ChatOpenAI  # The OpenAI model
from langchain.prompts import ChatPromptTemplate  # The prompt
from langchain.chains import LLMChain  # The llm chain

In [ ]:
# Initialize the language model
llm = ChatOpenAI(temperature=0.9)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

In [ ]:
# Combine the model and the prompt into a chain
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
product = "Queen size Sheet Set"
# chain.run(product)  # Deprecated use `invoke` instead of `run`
chain.invoke(product)

## SimpleSequentialChain

Sequential chains is another type of chains. The idea is to combine multiple chains where the output of the one chain is the input of the next chain.

There two type of sequential chains:
1. SimpleSequentialChain: Single input/outpur
2. SequentialChain: multiple inputs/outputs

Sequential chains run (invoke) a sequence of chains one after another.

In [ ]:
# Import the simple sequential chain (one input/output)
from langchain.chains import SimpleSequentialChain

In [ ]:
# Model
llm = ChatOpenAI(temperature=0.9)

# Prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}"
)

# Chain
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:
# Prompt template 2
second_promt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company: {company_name}"
)

# Chain 2
chain_two = LLMChain(llm=llm, prompt=second_promt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two],
    verbose=True
)

In [ ]:
overall_simple_chain.invoke(product)

## SequentialChain

Simple sequential chains work well when there is only a single input and a single output. But what when there are multiple inputs and multiple outputs?

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [ ]:
# Prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the followin review to english:"
    "\n\n{Review}"
)

# Chain 1: input = Review, output = English_Review
chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt,
    output_key="English_Review"
)

In [ ]:
# Prompt template 2: Summarize the english review
second_promt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)

# Chain 3: input = English_Review, output = summary
chain_two = LLMChain(
    llm=llm,
    prompt=second_promt,
    output_key="summary"
)

In [ ]:
# Prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# Chain 3: input = Review, output = language
chain_three = LLMChain(
    llm=llm, prompt=third_prompt,
    output_key="language"
)


In [ ]:

# Prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    """Write a follow up response to the following summary in the specified language:\
    \n\nSummary: {summary}\n\nLanguage: {language}"""
)
# Chain 4: input = summary, language and output = followup_message
chain_four = LLMChain(
    llm=llm, prompt=fourth_prompt,
    output_key="followup_message"
)


❗ For all the above subchains, the input key and the output key need to bw precise.

In [ ]:
# Overall chain: input = Review, output = English_Review, summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True
)

In [ ]:
review = df.Review[5]
overall_chain(review)